# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [3]:
stocks = pd.read_csv('sp_500_stocks.csv')
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [4]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2232359675523,
 'week52high': 140.77,
 'week52low': 57.14,
 'week52change': 0.6534119161523999,
 'sharesOutstanding': 17169389902,
 'float': 0,
 'avg10Volume': 98872870,
 'avg30Volume': 107814901,
 'day200MovingAvg': 117.22,
 'day50MovingAvg': 132.2,
 'employees': 0,
 'ttmEPS': 3.42,
 'ttmDividendRate': 0.8443342839537649,
 'dividendYield': 0.006314686749491614,
 'nextDividendDate': '0',
 'exDividendDate': '2020-10-24',
 'nextEarningsDate': '0',
 'peRatio': 38.34495412523061,
 'beta': 1.203973845452563,
 'maxChangePercent': 49.437951979368535,
 'year5ChangePercent': 4.902635629852563,
 'year2ChangePercent': 2.370790898977769,
 'year1ChangePercent': 0.6314414895105135,
 'ytdChangePercent': -0.03803962620257335,
 'month6ChangePercent': 0.3342543178194132,
 'month3ChangePercent': 0.10863794171999046,
 'month1ChangePercent': 0.009287247495076464,
 'day30ChangePercent': 0.009428772920233535,
 'day5ChangePercent': -0.007640313120527422}

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [5]:
data['year1ChangePercent']

0.6314414895105135

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [6]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [7]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,128.13,0.431433,N/A
1,AAL,16.76,-0.446522,N/A
2,AAP,170.21,0.107228,N/A
3,AAPL,129.27,0.626534,N/A
4,ABBV,113.71,0.35448,N/A
...,...,...,...,...
500,YUM,108.21,0.0402041,N/A
501,ZBH,163.05,0.0793127,N/A
502,ZBRA,408.82,0.633797,N/A
503,ZION,50.86,0.00927182,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [8]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,CARR,41.02,2.47625,N/A
1,FCX,32.61,1.48394,N/A
2,LB,46.37,1.34565,N/A
3,ALB,190.01,1.32977,N/A
4,PYPL,259.21,1.17783,N/A
5,NVDA,522.28,1.13536,N/A
6,ALGN,573.78,1.02505,N/A
7,LRCX,588.68,0.951584,N/A
8,PWR,79.54,0.948882,N/A
9,WST,310.22,0.928756,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [9]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:10000000
10000000


In [10]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

c:\users\pranit\anaconda3\envs\algotrade\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,CARR,41.02,2.47625,4780
1,FCX,32.61,1.48394,6012
2,LB,46.37,1.34565,4228
3,ALB,190.01,1.32977,1031
4,PYPL,259.21,1.17783,756
5,NVDA,522.28,1.13536,375
6,ALGN,573.78,1.02505,341
7,LRCX,588.68,0.951584,333
8,PWR,79.54,0.948882,2465
9,WST,310.22,0.928756,632


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [11]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'One-Year Price Return',
       'One-Year Return Percentile', 'Six-Month Price Return',
       'Six-Month Return Percentile', 'Three-Month Price Return',
       'Three-Month Return Percentile', 'One-Month Price Return',
       'One-Month Return Percentile', 'HQM Score'],
      dtype='object')

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [13]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        if hqm_dataframe.loc[row, f'{time_period} Price Return'] == None:
            hqm_dataframe.loc[row, f'{time_period} Price Return'] = 0

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

0       0.879208
1      0.0158416
2       0.544554
3       0.946535
4       0.825743
         ...    
500     0.431683
501     0.520792
502     0.936634
503     0.370297
504     0.625743
Name: One-Year Return Percentile, Length: 505, dtype: object
0      0.722772
1      0.679208
2      0.382178
3      0.675248
4      0.374257
         ...   
500    0.487129
501    0.457426
502    0.841584
503    0.875248
504    0.346535
Name: Six-Month Return Percentile, Length: 505, dtype: object
0      0.645545
1      0.726733
2      0.376238
3      0.419802
4      0.792079
         ...   
500    0.435644
501    0.481188
502    0.833663
503    0.952475
504    0.241584
Name: Three-Month Return Percentile, Length: 505, dtype: object
0       0.69703
1      0.158416
2      0.550495
3      0.390099
4      0.748515
         ...   
500    0.209901
501    0.639604
502    0.762376
503    0.942574
504     0.30297
Name: One-Month Return Percentile, Length: 505, dtype: object


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,132.57,N/A,0.425687,0.879208,0.366821,0.722772,0.217557,0.645545,0.0716824,0.69703,N/A
1,AAL,16.69,N/A,-0.438166,0.0158416,0.34326,0.679208,0.282616,0.726733,-0.0334701,0.158416,N/A
2,AAP,171.99,N/A,0.109126,0.544554,0.150919,0.382178,0.0885182,0.376238,0.0371722,0.550495,N/A
3,AAPL,129.90,N/A,0.646804,0.946535,0.338652,0.675248,0.107669,0.419802,0.00973253,0.390099,N/A
4,ABBV,114.13,N/A,0.351403,0.825743,0.144184,0.374257,0.357556,0.792079,0.0894836,0.748515,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.60,N/A,0.0399522,0.431683,0.205374,0.487129,0.112666,0.435644,-0.0225105,0.209901,N/A
501,ZBH,163.72,N/A,0.0822999,0.520792,0.190702,0.457426,0.13155,0.481188,0.0557683,0.639604,N/A
502,ZBRA,408.76,N/A,0.623685,0.936634,0.491127,0.841584,0.402237,0.833663,0.0910642,0.762376,N/A
503,ZION,50.84,N/A,0.00925646,0.370297,0.547315,0.875248,0.651273,0.952475,0.205703,0.942574,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [12]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [13]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)
hqm_dataframe = hqm_dataframe[:51]

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [14]:
portfolio_input()

Enter the value of your portfolio:1000000


In [15]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,98.19,199,0.444090,0.885149,0.147456,0.837624,0.221461,0.473267,0.093820,0.530693,0.681683
1,AAL,13.89,1411,-0.526494,0.0237624,-0.564540,0.0158416,0.509431,0.908911,0.172430,0.827723,0.444059
2,AAP,161.13,121,0.088066,0.578218,0.148378,0.839604,0.295700,0.643564,0.144608,0.742574,0.70099
3,AAPL,467.65,41,1.171724,0.992079,0.401695,0.968317,0.474900,0.887129,0.189840,0.879208,0.931683
4,ABBV,97.29,201,0.478770,0.89703,0.001711,0.629703,0.077880,0.19802,-0.024533,0.0693069,0.448515
5,ABC,104.97,186,0.163705,0.651485,0.100112,0.774257,0.241867,0.522772,0.066990,0.364356,0.578218
6,ABMD,305.78,64,0.546138,0.942574,0.874783,0.99802,0.645795,0.954455,0.145153,0.746535,0.910396
7,ABT,102.94,190,0.161073,0.647525,0.139130,0.823762,0.093917,0.233663,0.081815,0.457426,0.540594
8,ACN,237.88,82,0.197484,0.69505,0.085519,0.762376,0.279570,0.605941,0.067045,0.366337,0.607426
9,ADBE,453.91,43,0.532839,0.934653,0.196705,0.879208,0.252382,0.550495,0.006829,0.150495,0.628713


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [16]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [17]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [18]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [19]:
writer.save()